## Programs as word vectors

We are going to treat the programs students write and submit to our platform as word vectors. We are going to learn a representation of these programs. We are going to create a prediction problem and try to predict whether the program submitted with be considered as correct, i.e. the program pass the testcases specified by the lecturer. Creating a prediction problem to learn representations of the data is a useful approach. There is no easy way to represent these programs so we are going to learn these representations. 

In [1]:
import re
def remove_comments(text):
    return re.sub(re.compile('#.*?\n'), '', text)

In [2]:
program = '#!/usr/bin/env python\n\na = int(raw_input())\nb = int(raw_input())\n\nprint a + b\n\n\n'
remove_comments(program)

'\na = int(raw_input())\nb = int(raw_input())\n\nprint a + b\n\n\n'

In [3]:
program = '''
#!/usr/bin/env python
# read from input
a = int(raw_input()) # first
b = int(raw_input()) # second
print a + b
'''
remove_comments(program)

'\na = int(raw_input()) b = int(raw_input()) print a + b\n'

**Keras**

In [4]:
from keras.preprocessing.text import text_to_word_sequence

# text_to_word_sequence?

Using TensorFlow backend.


In [5]:
text_to_word_sequence(remove_comments(program), lower=True, split=' ')

['a', 'int', 'raw', 'input', 'b', 'int', 'raw', 'input', 'print', 'a', 'b']

By default in text_to_word_sequence the filters are:

> filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'

See the function signature

In [6]:
text_to_word_sequence(remove_comments(program), 
                      filters='\t\n',
                      lower=True, 
                      split=' ')

['a',
 '=',
 'int(raw_input())',
 'b',
 '=',
 'int(raw_input())',
 'print',
 'a',
 '+',
 'b']

It still does not represent a programming submission in a very comparable way to other submissions.

In [11]:
# https://keras.io/preprocessing/text/#tokenizer
from keras.preprocessing.text import Tokenizer

# Tokenizer?

# t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ', 
#              lower=True, split=' ', char_level=False)

In [7]:
# Run another notebook

# http://nbviewer.jupyter.org/gist/minrk/5491090/analysis.ipynb

#import io
#from IPython.nbformat import current

#def execute_notebook(nbfile):
#    with io.open(nbfile) as f:
#        nb = current.read(f, 'json')
#    ip = get_ipython()
#    for cell in nb.worksheets[0].cells:
#        if cell.cell_type != 'code':
#            continue
#        ip.run_cell(cell.input)

# execute_notebook("Tokenize Python programs.ipynb")

# get_tokens(program)

## Tokenize Python programs

Tokenize Python programs:
* https://docs.python.org/3/library/tokenize.html
* https://docs.python.org/3/library/token.html
* Code: https://github.com/python/cpython/blob/3.7/Lib/tokenize.py
* Code: https://github.com/python/cpython/blob/3.7/Lib/token.py
* C code: https://github.com/python/cpython/blob/master/Parser/tokenizer.c
* C library: https://github.com/python/cpython/blob/master/Include/token.h

In [1]:
import sys
from tokenize import generate_tokens
from StringIO import StringIO

In [2]:
from tokenize import LPAR, OP, Funny, AMPER, AMPEREQUAL, any, AT, Comment, COMMENT

print LPAR 
print OP
print AT
print Funny
print COMMENT
print Comment

7
51
50
((\*\*=?|>>=?|<<=?|<>|!=|//=?|[+\-*/%&|^=<>]=?|~)|[][(){}]|(\r?\n|[:;.,`@]))
53
#[^\r\n]*


**Command line**

In [3]:
!cat data/hello.py

def say_hello():
    print("Hello, World!")

say_hello()


In [4]:
!python -m tokenize data/hello.py

1,0-1,3:	NAME	'def'
1,4-1,13:	NAME	'say_hello'
1,13-1,14:	OP	'('
1,14-1,15:	OP	')'
1,15-1,16:	OP	':'
1,16-1,17:	NEWLINE	'\n'
2,0-2,4:	INDENT	'    '
2,4-2,9:	NAME	'print'
2,9-2,10:	OP	'('
2,10-2,25:	STRING	'"Hello, World!"'
2,25-2,26:	OP	')'
2,26-2,27:	NEWLINE	'\n'
3,0-3,1:	NL	'\n'
4,0-4,0:	DEDENT	''
4,0-4,9:	NAME	'say_hello'
4,9-4,10:	OP	'('
4,10-4,11:	OP	')'
4,11-4,12:	NEWLINE	'\n'
5,0-5,0:	ENDMARKER	''


**Programatically**

In [5]:
generate_tokens?

In [6]:
program_code = '''def say_hello():
    print("Hello, World!")

say_hello()'''

In [7]:
tokens = list(generate_tokens(StringIO(program_code).readline))

In [8]:
def print_tokens(tokens):
    for t in tokens:
        print (str(t[0]) + "," + str(t[2][1]) + "-" + str(t[2][0]) + ',' + str(t[3][1]) + ':\t' + 
               str(t[0]) + '\t' + # id_to_token[t[0]]
               repr(str(t[1])))

In [9]:
print_tokens(tokens)

1,0-1,3:	1	'def'
1,4-1,13:	1	'say_hello'
51,13-1,14:	51	'('
51,14-1,15:	51	')'
51,15-1,16:	51	':'
4,16-1,17:	4	'\n'
5,0-2,4:	5	'    '
1,4-2,9:	1	'print'
51,9-2,10:	51	'('
3,10-2,25:	3	'"Hello, World!"'
51,25-2,26:	51	')'
4,26-2,27:	4	'\n'
54,0-3,1:	54	'\n'
6,0-4,0:	6	''
1,0-4,9:	1	'say_hello'
51,9-4,10:	51	'('
51,10-4,11:	51	')'
0,0-5,0:	0	''


In [10]:
# List of:
# Token code, Part of the line being analysed, FROM (line, character), TO (line, character), Complete line of code
tokens

[(1, 'def', (1, 0), (1, 3), 'def say_hello():\n'),
 (1, 'say_hello', (1, 4), (1, 13), 'def say_hello():\n'),
 (51, '(', (1, 13), (1, 14), 'def say_hello():\n'),
 (51, ')', (1, 14), (1, 15), 'def say_hello():\n'),
 (51, ':', (1, 15), (1, 16), 'def say_hello():\n'),
 (4, '\n', (1, 16), (1, 17), 'def say_hello():\n'),
 (5, '    ', (2, 0), (2, 4), '    print("Hello, World!")\n'),
 (1, 'print', (2, 4), (2, 9), '    print("Hello, World!")\n'),
 (51, '(', (2, 9), (2, 10), '    print("Hello, World!")\n'),
 (3, '"Hello, World!"', (2, 10), (2, 25), '    print("Hello, World!")\n'),
 (51, ')', (2, 25), (2, 26), '    print("Hello, World!")\n'),
 (4, '\n', (2, 26), (2, 27), '    print("Hello, World!")\n'),
 (54, '\n', (3, 0), (3, 1), '\n'),
 (6, '', (4, 0), (4, 0), 'say_hello()'),
 (1, 'say_hello', (4, 0), (4, 9), 'say_hello()'),
 (51, '(', (4, 9), (4, 10), 'say_hello()'),
 (51, ')', (4, 10), (4, 11), 'say_hello()'),
 (0, '', (5, 0), (5, 0), '')]

## Abstract Syntax Trees

https://docs.python.org/3/library/ast.html

In [11]:
import ast

In [25]:
program_code

'def say_hello():\n    print("Hello, World!")\n\nsay_hello()'

In [12]:
program_ast = ast.parse(program_code)

In [13]:
program_ast

In [14]:
ast.dump(program_ast)

"Module(body=[FunctionDef(name='say_hello', args=arguments(args=[], vararg=None, kwarg=None, defaults=[]), body=[Print(dest=None, values=[Str(s='Hello, World!')], nl=True)], decorator_list=[]), Expr(value=Call(func=Name(id='say_hello', ctx=Load()), args=[], keywords=[], starargs=None, kwargs=None))])"

In [15]:
program_ast.body

[<_ast.FunctionDef at 0x108a1a790>, <_ast.Expr at 0x108a1ac50>]

In [16]:
for statement in program_ast.body:
    print ast.dump(statement), '\n'

FunctionDef(name='say_hello', args=arguments(args=[], vararg=None, kwarg=None, defaults=[]), body=[Print(dest=None, values=[Str(s='Hello, World!')], nl=True)], decorator_list=[]) 

Expr(value=Call(func=Name(id='say_hello', ctx=Load()), args=[], keywords=[], starargs=None, kwargs=None)) 



In [17]:
class FunctionCallVisitor(ast.NodeVisitor):
    def visit_Call(self, node):
        print ast.dump(node)

FunctionCallVisitor().visit(program_ast)

Call(func=Name(id='say_hello', ctx=Load()), args=[], keywords=[], starargs=None, kwargs=None)


In [18]:
divisors_code = '''def divisors(n):
    for d in range(1, n + 1):
        if n % d == 0:
            print d'''

In [19]:
ast.dump(ast.parse(divisors_code))

"Module(body=[FunctionDef(name='divisors', args=arguments(args=[Name(id='n', ctx=Param())], vararg=None, kwarg=None, defaults=[]), body=[For(target=Name(id='d', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[Num(n=1), BinOp(left=Name(id='n', ctx=Load()), op=Add(), right=Num(n=1))], keywords=[], starargs=None, kwargs=None), body=[If(test=Compare(left=BinOp(left=Name(id='n', ctx=Load()), op=Mod(), right=Name(id='d', ctx=Load())), ops=[Eq()], comparators=[Num(n=0)]), body=[Print(dest=None, values=[Name(id='d', ctx=Load())], nl=True)], orelse=[])], orelse=[])], decorator_list=[])])"

Visualize the tree:

* https://github.com/hchasestevens/show_ast
* http://www.pycs.net/users/0000445/stories/3.html


**Vocabulary**

In [29]:
token_to_id = {
    'ENDMARKER': 0,
    'NAME': 1,
    'NUMBER': 2,
    'STRING': 3,
    'NEWLINE': 4,
    'INDENT': 5,
    'DEDENT': 6,
    'LPAR': 7,
    'RPAR': 8,
    'LSQB': 9,
    'RSQB': 10,
    'COLON': 11,
    'COMMA': 12,
    'SEMI': 13,
    'PLUS': 14,
    'MINUS': 15,
    'STAR': 16,
    'SLASH': 17,
    'VBAR': 18,
    'AMPER': 19,
    'LESS': 20,
    'GREATER': 21,
    'EQUAL': 22,
    'DOT': 23,
    'PERCENT': 24,
    'BACKQUOTE': 25,
    'LBRACE': 26,
    'RBRACE': 27,
    'EQEQUAL': 28,
    'NOTEQUAL': 29,
    'LESSEQUAL': 30,
    'GREATEREQUAL': 31,
    'TILDE': 32,
    'CIRCUMFLEX': 33,
    'LEFTSHIFT': 34,
    'RIGHTSHIFT': 35,
    'DOUBLESTAR': 36,
    'PLUSEQUAL': 37,
    'MINEQUAL': 38,
    'STAREQUAL': 39,
    'SLASHEQUAL': 40,
    'PERCENTEQUAL': 41,
    'AMPEREQUAL': 42,
    'VBAREQUAL': 43,
    'CIRCUMFLEXEQUAL': 44,
    'LEFTSHIFTEQUAL': 45,
    'RIGHTSHIFTEQUAL': 46,
    'DOUBLESTAREQUAL': 47,
    'DOUBLESLASH': 48,
    'DOUBLESLASHEQUAL': 49,
    'AT': 50,
    'ATEQUAL': 51,
    'OP': 52,
    'COMMENT': 53,
    'NL': 54,
    'RARROW': 55,
    'ERRORTOKEN': 56,
    'N_TOKENS': 57,
    'NT_OFFSET': 256,
}

In [32]:
id_to_token = { v:k for k, v in token_to_id.iteritems() }

In [33]:
id_to_token[20]

'LESS'

In [96]:
vocabulary = token_to_id.keys()

In [97]:
keywords = [ 'and', 'del', 'from', 'not', 'while', 'as', 'elif', 'global', 'or', 'with', 
            'assert', 'else', 'if', 'pass', 'yield', 'break', 'except', 'import', 'print', 
            'class', 'exec', 'in', 'raise', 'continue', 'finally', 'is', 'return', 'def', 
            'for', 'lambda', 'try']

built_functions = ['abs', 'divmod', 'input', 'open', 'staticmethod', 'all', 'enumerate', 'int', 
                   'ord', 'str', 'any', 'eval', 'isinstance', 'pow', 'sum', 'basestring', 'execfile', 
                   'issubclass', 'print', 'super', 'bin', 'file', 'iter', 'property', 'tuple', 'bool', 
                   'filter', 'len', 'range', 'type', 'bytearray', 'float', 'list', 'raw_input', 'unichr', 
                   'callable', 'format', 'locals', 'reduce', 'unicode', 'chr', 'frozenset', 'long', 'reload', 
                   'vars', 'classmethod', 'getattr', 'map', 'repr', 'xrange', 'cmp', 'globals', 'max', 
                   'reversed', 'zip', 'compile', 'hasattr', 'memoryview', 'round', '__import__', 'complex', 
                   'hash', 'min', 'set', 'delattr', 'help', 'next', 'setattr', 'dict', 'hex', 'object', 
                   'slice', 'dir', 'id', 'oct', 'sorted']

operators = ['+', '-', '*', '**', '/', '//', '%', '<<', '>>', '&', '|', '^', '~', '<', '>', 
             '<=', '>=', '==', '!=', '<>']

delimiters = ['(', ')', '[', ']', '{', '}', '@', ',', ':', '.', '`', '=', ';', '+=', '-=', '*=', 
              '/=', '//=', '%=', '&=', '|=', '^=', '>>=', '<<=', '**=',
              '\'', '"', '#', '\\', '$', '?']

In [98]:
vocabulary.extend(keywords)
vocabulary.extend(built_functions)
vocabulary.extend(operators)
vocabulary.extend(delimiters)
# vocabulary

In [99]:
# len(vocabulary)

In [100]:
# vocabulary[0]

In [101]:
# vocabulary[100]

In [102]:
vocab_to_id = { k:v for k, v in set(enumerate(vocabulary)) }

In [103]:
# vocab_to_id[0], vocab_to_id[216]

In [104]:
id_to_vocab = { v:k + 2 for k,v in vocab_to_id.iteritems() }
id_to_vocab['<PAD>'] = 0
id_to_vocab['<UNK>'] = 1

In [105]:
# id_to_vocab['<PAD>']

In [106]:
# id_to_vocab['>']

In [107]:
# id_to_vocab['while']

In [108]:
# for key, value in sorted(id_to_vocab.iteritems(), key=lambda (k,v): (v,k)):
#     print "%s: %s" % (key, value)

In [112]:
def get_tokens(code):

    try:
        tokens = generate_tokens(StringIO(code).readline)
        # return [row[0] for row in list(tokens)]
        
        _ids = []
        
        for row in list(tokens):
            
            print row
            
            token_num, token_value, line_pos_from, line_pos_to, line = row
            
            if token_num in id_to_token:
                
                print token_num
                
                if token_num == 1 and token_value in id_to_vocab: # id_to_token[token_num] == 'NAME'
                    _id = id_to_vocab[token_value]
                else:
                    _id = id_to_vocab[id_to_token[token_num]]
            else:
                _id = id_to_vocab['<UNK>']
                
            _ids.append(_id)
            
        return _ids

    except:
        print "Unexpected error:", sys.exc_info()[0]
        return None

In [113]:
program_a

'\ndef get_name(name):\n    print "Hi %s" % (name)\nget_name(\'David\')\n'

In [114]:
get_tokens(program_a)

(54, '\n', (1, 0), (1, 1), '\n')
54
(1, 'def', (2, 0), (2, 3), 'def get_name(name):\n')
1
(1, 'get_name', (2, 4), (2, 12), 'def get_name(name):\n')
1
(51, '(', (2, 12), (2, 13), 'def get_name(name):\n')
51
(1, 'name', (2, 13), (2, 17), 'def get_name(name):\n')
1
(51, ')', (2, 17), (2, 18), 'def get_name(name):\n')
51
(51, ':', (2, 18), (2, 19), 'def get_name(name):\n')
51
(4, '\n', (2, 19), (2, 20), 'def get_name(name):\n')
4
(5, '    ', (3, 0), (3, 4), '    print "Hi %s" % (name)\n')
5
(1, 'print', (3, 4), (3, 9), '    print "Hi %s" % (name)\n')
1
(3, '"Hi %s"', (3, 10), (3, 17), '    print "Hi %s" % (name)\n')
3
(51, '%', (3, 18), (3, 19), '    print "Hi %s" % (name)\n')
51
(51, '(', (3, 20), (3, 21), '    print "Hi %s" % (name)\n')
51
(1, 'name', (3, 21), (3, 25), '    print "Hi %s" % (name)\n')
1
(51, ')', (3, 25), (3, 26), '    print "Hi %s" % (name)\n')
51
(4, '\n', (3, 26), (3, 27), '    print "Hi %s" % (name)\n')
4
(6, '', (4, 0), (4, 0), "get_name('David')\n")
6
(1, 'get_name'

[13,
 88,
 30,
 55,
 30,
 55,
 55,
 15,
 31,
 110,
 53,
 55,
 55,
 30,
 55,
 15,
 35,
 30,
 55,
 53,
 55,
 15,
 3]

In [6]:
program_sum = '''
#!/usr/bin/env python

# read from input
a = int(raw_input()) # first
b = int(raw_input()) # second

print a + b'''

In [7]:
!cat data/sum.py

#!/usr/bin/env python

# read from input
a = int(raw_input()) # first
b = int(raw_input()) # second

print a + b


In [8]:
!python -m tokenize data/sum.py

1,0-1,21:	COMMENT	'#!/usr/bin/env python'
1,21-1,22:	NL	'\n'
2,0-2,1:	NL	'\n'
3,0-3,17:	COMMENT	'# read from input'
3,17-3,18:	NL	'\n'
4,0-4,1:	NAME	'a'
4,2-4,3:	OP	'='
4,4-4,7:	NAME	'int'
4,7-4,8:	OP	'('
4,8-4,17:	NAME	'raw_input'
4,17-4,18:	OP	'('
4,18-4,19:	OP	')'
4,19-4,20:	OP	')'
4,21-4,28:	COMMENT	'# first'
4,28-4,29:	NEWLINE	'\n'
5,0-5,1:	NAME	'b'
5,2-5,3:	OP	'='
5,4-5,7:	NAME	'int'
5,7-5,8:	OP	'('
5,8-5,17:	NAME	'raw_input'
5,17-5,18:	OP	'('
5,18-5,19:	OP	')'
5,19-5,20:	OP	')'
5,21-5,29:	COMMENT	'# second'
5,29-5,30:	NEWLINE	'\n'
6,0-6,1:	NL	'\n'
7,0-7,5:	NAME	'print'
7,6-7,7:	NAME	'a'
7,8-7,9:	OP	'+'
7,10-7,11:	NAME	'b'
7,11-7,12:	NEWLINE	'\n'
8,0-8,0:	ENDMARKER	''


In [20]:
program_score = '''
n = input()
i = 0
away_score = (away_goals * 3) + away_points
home_score = (home_goals * 3) + home_points 
while n < 4:
    n = input()
    if home_score > away_score :
        print "home win"
    elif home_score < away_score :
        print "away win"
    else:
        print "draw"
    i = i + 1
'''

In [24]:
!cat data/score.py

n = input()
i = 0
away_score = (away_goals * 3) + away_points
home_score = (home_goals * 3) + home_points 
while n < 4:
    n = input()
    if home_score > away_score :
        print "home win"
    elif home_score < away_score :
        print "away win"
    else:
        print "draw"
    i = i + 1


In [22]:
!python -m tokenize data/score.py

1,0-1,1:	NAME	'n'
1,2-1,3:	OP	'='
1,4-1,9:	NAME	'input'
1,9-1,10:	OP	'('
1,10-1,11:	OP	')'
1,11-1,12:	NEWLINE	'\n'
2,0-2,1:	NAME	'i'
2,2-2,3:	OP	'='
2,4-2,5:	NUMBER	'0'
2,5-2,6:	NEWLINE	'\n'
3,0-3,10:	NAME	'away_score'
3,11-3,12:	OP	'='
3,13-3,14:	OP	'('
3,14-3,24:	NAME	'away_goals'
3,25-3,26:	OP	'*'
3,27-3,28:	NUMBER	'3'
3,28-3,29:	OP	')'
3,30-3,31:	OP	'+'
3,32-3,43:	NAME	'away_points'
3,43-3,44:	NEWLINE	'\n'
4,0-4,10:	NAME	'home_score'
4,11-4,12:	OP	'='
4,13-4,14:	OP	'('
4,14-4,24:	NAME	'home_goals'
4,25-4,26:	OP	'*'
4,27-4,28:	NUMBER	'3'
4,28-4,29:	OP	')'
4,30-4,31:	OP	'+'
4,32-4,43:	NAME	'home_points'
4,44-4,45:	NEWLINE	'\n'
5,0-5,5:	NAME	'while'
5,6-5,7:	NAME	'n'
5,8-5,9:	OP	'<'
5,10-5,11:	NUMBER	'4'
5,11-5,12:	OP	':'
5,12-5,13:	NEWLINE	'\n'
6,0-6,4:	INDENT	'    '
6,4-6,5:	NAME	'n'
6,6-6,7:	OP	'='
6,8-6,13:	NAME	'input'
6,13-6,14:	OP	'('
6,14-6,15:	OP	')'
6,15-6,16:	NEWLINE	'\n'
7,4-7,6:	NAME	'if'
7,7-7,17:	NAME	'home_score'
7,18-7,19:	OP	'>'
7,20-7,30:	NAME	'away_score'
7,31-7,32